# Kite Connect Testing Notebook

Test suite for Kite API integration, position analysis, and trading operations.

## 1. Import Dependencies and Initialize Kite

In [69]:
# In your main_strategy.py
from pprint import pprint
import pandas as pd
import numpy as np
from datetime import datetime, date

In [70]:
# Reload the module to get fresh kite object with new token
import importlib
import load_kite_from_access
#importlib.reload(load_kite_from_access)
kite = load_kite_from_access.kite

# Check kite status and get UserID
print(f"Kite object: {kite}")
print(f"Type: {type(kite)}")
if kite:
    try:
        profile = kite.profile()
        print(f"\n✅ UserID: {profile['user_id']}")
        print(f"✅ User Name: {profile['user_name']}")
        print(f"✅ Email: {profile['email']}")
    except Exception as e:
        print(f"Error getting profile: {e}")
else:
    print("\n❌ Kite object is None - session not established")
    print("Please ensure you have run config.py to generate access token")

Kite object: <kiteconnect.connect.KiteConnect object at 0x00000204ABC30EC0>
Type: <class 'kiteconnect.connect.KiteConnect'>
Error getting profile: Incorrect `api_key` or `access_token`.


## 3. Fetch and Group Instruments (NSE + NFO)

In [ ]:
try:
    # 1. Fetch instruments from NSE (Equities & Indices) and NFO (Futures & Options)
    print("Fetching instruments from NSE...")
    instruments_nse = kite.instruments("NSE")
    
    print("Fetching instruments from NFO...")
    instruments_nfo = kite.instruments("NFO")
    
    print("Fetching instruments from MCX...")
    instruments_mcx = kite.instruments("MCX")

    # 2. Combine and convert to DataFrame
    instruments_all = instruments_nse + instruments_nfo + instruments_mcx
    
    df_all = pd.DataFrame(instruments_all)
    
    
    print(f"\nTotal Instruments Fetched: {len(df_all):,}")
    print(f"  - NSE: {len(instruments_nse):,}")
    print(f"  - NFO: {len(instruments_nfo):,}")
    print(f"\nInstrument Types: {sorted(df_all['instrument_type'].unique())}")
    print(f"Segments: {sorted(df_all['segment'].unique())}")
    
    # 3. Group instruments by type
    # Equity (EQ) - from NSE
    df_equity = df_all[(df_all['instrument_type'] == 'EQ') & (df_all['segment'] == 'NSE')].copy()
    
    # Index - INDICES segment
    df_index = df_all[df_all['segment'] == 'INDICES'].copy()
    
    # Futures (FUT) - from NFO
    df_futures = df_all[df_all['instrument_type'] == 'FUT'].copy()
    
    # # Options - Call (CE)
    # df_options_ce = df_all[df_all['instrument_type'] == 'CE'].copy()
    
    # # Options - Put (PE)
    # df_options_pe = df_all[df_all['instrument_type'] == 'PE'].copy()
    
    # 4. Display summary
    print("\n" + "="*60)
    print("INSTRUMENT GROUPING SUMMARY")
    print("="*60)
    print(f"Equity (EQ):          {len(df_equity):,} instruments")
    print(f"Indices:              {len(df_index):,} instruments")
    # print(f"Futures (FUT):        {len(df_futures):,} instruments")
    # print(f"Options - Call (CE):  {len(df_options_ce):,} instruments")
    # print(f"Options - Put (PE):   {len(df_options_pe):,} instruments")
    print("="*60)
    
    # 5. Show samples of each type
    print("\n EQUITY SAMPLE (Top 5):")
    if len(df_equity) > 0:
        print(df_equity[['tradingsymbol', 'name', 'exchange', 'instrument_type']].head())
    
    print("\n INDEX SAMPLE (Top 5):")
    if len(df_index) > 0:
        print(df_index[['tradingsymbol', 'name', 'exchange']].head())
    
    # print("\n FUTURES SAMPLE (Top 5):")
    # if len(df_futures) > 0:
    #     print(df_futures[['tradingsymbol', 'name', 'expiry', 'lot_size', 'instrument_type']].head())
    
    # print("\n OPTIONS CE SAMPLE (Top 5):")
    # if len(df_options_ce) > 0:
    #     print(df_options_ce[['tradingsymbol', 'name', 'strike', 'expiry', 'lot_size']].head())
    
    # print("\n OPTIONS PE SAMPLE (Top 5):")
    # if len(df_options_pe) > 0:
    #     print(df_options_pe[['tradingsymbol', 'name', 'strike', 'expiry', 'lot_size']].head())
    
    print("\n✅ All instruments grouped and stored in pandas DataFrames:")

except Exception as e:
    print(f"Error occurred: {e}")
    import traceback
    traceback.print_exc()

Fetching instruments from NSE...
Fetching instruments from NFO...
Fetching instruments from MCX...

Total Instruments Fetched: 121,771
  - NSE: 9,202
  - NFO: 38,795

Instrument Types: ['CE', 'EQ', 'FUT', 'PE']
Segments: ['INDICES', 'MCX-FUT', 'MCX-OPT', 'NFO-FUT', 'NFO-OPT', 'NSE']

INSTRUMENT GROUPING SUMMARY
Equity (EQ):          9,068 instruments
Indices:              145 instruments

 EQUITY SAMPLE (Top 5):
    tradingsymbol                       name exchange instrument_type
134   GOLDSTAR-SM             GOLDSTAR POWER      NSE              EQ
135    21STCENMGM  21ST CENTURY MGMT SERVICE      NSE              EQ
136      AARTIIND           AARTI INDUSTRIES      NSE              EQ
137           ABB                  ABB INDIA      NSE              EQ
138    656KA30-SG          SDL KA 6.56% 2030      NSE              EQ

 INDEX SAMPLE (Top 5):
       tradingsymbol               name exchange
0           NIFTY 50           NIFTY 50      NSE
1   NIFTY MIDCAP 100   NIFTY MIDCAP 100   

## Standardise Date and Time


In [16]:

# Standardize and format all dataframes for consistency
print("="*80)
print("STANDARDIZING DATA FORMATS")
print("="*80)

# Helper function to standardize datetime columns
def standardize_dataframe(df, name="DataFrame"):
    df_clean = df.copy()
    
    # Convert date columns to datetime64
    date_columns = ['expiry', 'last_date']
    for col in date_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
    
    # Ensure numeric columns are proper types
    numeric_columns = ['strike', 'tick_size', 'lot_size']
    for col in numeric_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    # String columns should be proper strings (including tokens)
    string_columns = ['tradingsymbol', 'name', 'exchange', 'segment', 'instrument_type', 
                      'instrument_token', 'exchange_token']
    for col in string_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(str)
    
    print(f"✅ {name}: {len(df_clean):,} rows standardized")
    return df_clean

# Standardize all NSE/NFO dataframes
print("\n📊 NSE/NFO DataFrames:")
df_all = standardize_dataframe(df_all, "df_all")
df_equity = standardize_dataframe(df_equity, "df_equity")
df_index = standardize_dataframe(df_index, "df_index")
# df_futures = standardize_dataframe(df_futures, "df_futures")
# df_options_ce = standardize_dataframe(df_options_ce, "df_options_ce")
# df_options_pe = standardize_dataframe(df_options_pe, "df_options_pe")

STANDARDIZING DATA FORMATS

📊 NSE/NFO DataFrames:
✅ df_all: 121,771 rows standardized
✅ df_equity: 9,068 rows standardized
✅ df_index: 145 rows standardized


In [17]:
# Standardize MCX dataframe
print("\n📊 MCX DataFrames:")
df_mcx = pd.DataFrame(instruments_mcx)
df_mcx = standardize_dataframe(df_mcx, "df_mcx")

print("\n" + "="*80)
print("DATA STANDARDIZATION COMPLETE")
print("="*80)
print("All dataframes now have:")
print("  - DateTime columns in datetime64 format")
print("  - Numeric columns (strike, tick_size, lot_size) in proper numeric types")
print("  - String columns (including instrument_token, exchange_token) as strings")
print("  - Ready for Parquet export!")



📊 MCX DataFrames:
✅ df_mcx: 73,774 rows standardized

DATA STANDARDIZATION COMPLETE
All dataframes now have:
  - DateTime columns in datetime64 format
  - Numeric columns (strike, tick_size, lot_size) in proper numeric types
  - String columns (including instrument_token, exchange_token) as strings
  - Ready for Parquet export!


## 4. Export Instruments to CSV Files

In [ ]:
# Save all DataFrames to Parquet files for efficient storage
import os

# Create a folder for the data
output_folder = "instruments_data"
os.makedirs(output_folder, exist_ok=True)

# Save each DataFrame to Parquet (data already standardized in previous cell)
files_created = []

# NSE + NFO instruments
df_all.to_parquet(f"{output_folder}/df_all.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/df_all.parquet")

df_equity.to_parquet(f"{output_folder}/equity.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/equity.parquet")

df_index.to_parquet(f"{output_folder}/indices.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/indices.parquet")

df_futures.to_parquet(f"{output_folder}/futures.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/futures.parquet")

df_options_ce.to_parquet(f"{output_folder}/options_ce.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/options_ce.parquet")

df_options_pe.to_parquet(f"{output_folder}/options_pe.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/options_pe.parquet")

# MCX dataframes (already created and standardized in previous cell)
df_mcx.to_parquet(f"{output_folder}/mcx_all.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/mcx_all.parquet")

df_mcx_futures.to_parquet(f"{output_folder}/mcx_futures.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/mcx_futures.parquet")

df_mcx_ce.to_parquet(f"{output_folder}/mcx_options_ce.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/mcx_options_ce.parquet")

df_mcx_pe.to_parquet(f"{output_folder}/mcx_options_pe.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/mcx_options_pe.parquet")

print("✅ Parquet files created successfully!")
print(f"\n📊 Summary:")
print(f"   - NSE/NFO Total: {len(df_all):,} instruments")
print(f"   - MCX Total: {len(df_mcx):,} instruments")
print(f"   - MCX Futures: {len(df_mcx_futures):,} instruments")
print(f"   - MCX CE: {len(df_mcx_ce):,} instruments")
print(f"   - MCX PE: {len(df_mcx_pe):,} instruments")

print("\n📁 Files saved to:")
for file in files_created:
    full_path = os.path.abspath(file)
    file_size = os.path.getsize(full_path) / (1024 * 1024)  # Size in MB
    print(f"   - {file} ({file_size:.2f} MB)")
    
print("\n💡 To view in Python:")
print("   import pandas as pd")
print("   df = pd.read_parquet('instruments_data/equity.parquet')")

## 5. Create Instrument Tree (Nested Dictionary)

In [ ]:
# Create a nested dictionary: { name: { type: dataframe } }
# This groups by name first, then by instrument_type within each name
instrument_tree = {
    name: {inst_type: data for inst_type, data in name_group.groupby('instrument_type')}
    for name, name_group in df_all.groupby('name')
}



In [ ]:
print(f"✅ Created instrument_tree with {len(instrument_tree)} unique instrument names")
print(f"Example: instrument_tree['BSE'] contains: {list(instrument_tree.get('BSE', {}).keys())}")

# Usage:
BSE = instrument_tree['BSE']['FUT']
pprint(BSE)

## 5. Option Chain Analysis

In [43]:
# Create lookup dictionary for symbols with options/futures
# Format: { 'SYMBOL_NAME': { 'exchange': 'NFO/MCX', 'options_expiries': [...], 'futures_expiries': [...] } }
# Purpose: Track which exchange has derivatives for each symbol + separate expiry tracking

symbol_lookup = {}

print("Building symbol lookup from df_all...")

if 'df_all' in locals():
    # Get all symbols that have derivatives (FUT, CE, or PE)
    df_derivatives = df_all[df_all['instrument_type'].isin(['FUT', 'CE', 'PE'])]
    
    for name, group in df_derivatives.groupby('name'):
        # Get the exchange where derivatives are traded for this symbol
        exchanges = group['exchange'].unique()
        
        # Logic: Derivatives are on NFO (equity/index) or MCX (commodities)
        if 'NFO' in exchanges:
            derivatives_exchange = 'NFO'
        elif 'MCX' in exchanges:
            derivatives_exchange = 'MCX'
        else:
            derivatives_exchange = exchanges[0]  # Fallback
        
        # Extract expiries separately for options and futures
        options_data = group[group['instrument_type'].isin(['CE', 'PE'])]
        futures_data = group[group['instrument_type'] == 'FUT']
        
        options_expiries = sorted(options_data['expiry'].dropna().unique())
        futures_expiries = sorted(futures_data['expiry'].dropna().unique())
        
        symbol_lookup[name] = {
            'exchange': derivatives_exchange,
            'options_expiries': options_expiries,
            'futures_expiries': futures_expiries
        }
    
    print(f"\n✅ Symbol Lookup Created for {len(symbol_lookup):,} symbols with derivatives")
    
    # --- TEST VERIFICATION ---
    print("-" * 70)
    print("TESTING LOOKUP:")
    test_symbols = ['GOLDM', 'CRUDEOIL', 'ADANIENT', 'NIFTY', 'RELIANCE']
    
    for sym in test_symbols:
        if sym in symbol_lookup:
            entry = symbol_lookup[sym]
            opt_count = len(entry['options_expiries'])
            fut_count = len(entry['futures_expiries'])
            
            opt_next = entry['options_expiries'][0].date() if opt_count > 0 else "None"
            fut_next = entry['futures_expiries'][0].date() if fut_count > 0 else "None"
            
            print(f"✅ {sym:<10} | Exch: {entry['exchange']:<5}")
            print(f"   Options: {opt_count} expiries (Next: {opt_next})")
            print(f"   Futures: {fut_count} expiries (Next: {fut_next})")
        else:
            print(f"❌ {sym:<10} | No derivatives found")
    print("-" * 70)
else:
    print("❌ df_all not found. Please run the instrument fetch cell first.")

Building symbol lookup from df_all...

✅ Symbol Lookup Created for 241 symbols with derivatives
----------------------------------------------------------------------
TESTING LOOKUP:
✅ GOLDM      | Exch: MCX  
   Options: 2 expiries (Next: 2026-02-26)
   Futures: 6 expiries (Next: 2026-02-05)
✅ CRUDEOIL   | Exch: MCX  
   Options: 3 expiries (Next: 2026-02-17)
   Futures: 6 expiries (Next: 2026-02-19)
✅ ADANIENT   | Exch: NFO  
   Options: 3 expiries (Next: 2026-02-24)
   Futures: 3 expiries (Next: 2026-02-24)
✅ NIFTY      | Exch: NFO  
   Options: 18 expiries (Next: 2026-02-03)
   Futures: 3 expiries (Next: 2026-02-24)
✅ RELIANCE   | Exch: NFO  
   Options: 3 expiries (Next: 2026-02-24)
   Futures: 3 expiries (Next: 2026-02-24)
----------------------------------------------------------------------


In [44]:
symbol_lookup

{'360ONE': {'exchange': 'NFO',
  'options_expiries': [Timestamp('2026-02-24 00:00:00'),
   Timestamp('2026-03-30 00:00:00'),
   Timestamp('2026-04-28 00:00:00')],
  'futures_expiries': [Timestamp('2026-02-24 00:00:00'),
   Timestamp('2026-03-30 00:00:00'),
   Timestamp('2026-04-28 00:00:00')]},
 'ABB': {'exchange': 'NFO',
  'options_expiries': [Timestamp('2026-02-24 00:00:00'),
   Timestamp('2026-03-30 00:00:00'),
   Timestamp('2026-04-28 00:00:00')],
  'futures_expiries': [Timestamp('2026-02-24 00:00:00'),
   Timestamp('2026-03-30 00:00:00'),
   Timestamp('2026-04-28 00:00:00')]},
 'ABCAPITAL': {'exchange': 'NFO',
  'options_expiries': [Timestamp('2026-02-24 00:00:00'),
   Timestamp('2026-03-30 00:00:00'),
   Timestamp('2026-04-28 00:00:00')],
  'futures_expiries': [Timestamp('2026-02-24 00:00:00'),
   Timestamp('2026-03-30 00:00:00'),
   Timestamp('2026-04-28 00:00:00')]},
 'ADANIENSOL': {'exchange': 'NFO',
  'options_expiries': [Timestamp('2026-02-24 00:00:00'),
   Timestamp('2026-0

In [45]:
# Function to get all available expiries for a symbol
def get_all_expiries(symbol, derivative_type='options'):
    """
    Get all available expiry dates for a given symbol using the pre-computed lookup table.
    
    """
    if symbol not in symbol_lookup:
        print(f"❌ Base Symbol '{symbol}' not found in lookup table")
        return []
    
    # Return the appropriate expiry list
    if derivative_type == 'futures':
        return symbol_lookup[symbol]['futures_expiries']
    else:
        return symbol_lookup[symbol]['options_expiries']

# Example usage help
print("✅ get_all_expiries(symbol, derivative_type='options') updated.")
print("   Usage: get_all_expiries('GOLDM', 'options')  # Get option expiries")
print("          get_all_expiries('GOLDM', 'futures')  # Get futures expiries")

✅ get_all_expiries(symbol, derivative_type='options') updated.
   Usage: get_all_expiries('GOLDM', 'options')  # Get option expiries
          get_all_expiries('GOLDM', 'futures')  # Get futures expiries


In [46]:
# Test the simplified function
print("Testing get_all_expiries for GOLDM:")
print("\n📅 Options Expiries:")

options_exp = get_all_expiries('GOLDM', 'options')
for exp in options_exp:
    print(f"   {exp.date()}")

print("\n📅 Futures Expiries:")
futures_exp = get_all_expiries('GOLDM', 'futures')
for exp in futures_exp:
    print(f"   {exp.date()}")

print("\n" + "="*50)
print("Testing with NIFTY:")
print(f"Options: {len(get_all_expiries('NIFTY', 'options'))} expiries")
print(f"Futures: {len(get_all_expiries('NIFTY', 'futures'))} expiries")

Testing get_all_expiries for GOLDM:

📅 Options Expiries:
   2026-02-26
   2026-03-26

📅 Futures Expiries:
   2026-02-05
   2026-03-05
   2026-04-03
   2026-05-05
   2026-06-05
   2026-07-03

Testing with NIFTY:
Options: 18 expiries
Futures: 3 expiries


In [55]:
def build_option_chain(symbol, expiry=None, enrich=False):
    """
    Build option chain for a symbol using symbol_lookup and pre-loaded instrument data.
    No Kite API calls needed - uses cached df_all/df_mcx DataFrames.
    
    Args:
        symbol (str): Base symbol (e.g. 'NIFTY', 'GOLDM')
        expiry (str/date, optional): Expiry date. Defaults to nearest upcoming options expiry.
        enrich (bool): If True, includes market data (LTP, OI, volume). Requires kite object.
        
    Returns:
        DataFrame: Option chain with memory-optimized dtypes [tradingsymbol, strike, instrument_type, expiry, lot_size, etc]
    """
    # 1. Validate symbol exists in lookup
    if symbol not in symbol_lookup:
        print(f"❌ Symbol '{symbol}' not found in symbol_lookup")
        return pd.DataFrame()
    
    # 2. Get exchange and options expiries from lookup
    exchange = symbol_lookup[symbol]['exchange']
    options_expiries = symbol_lookup[symbol]['options_expiries']
    
    if len(options_expiries) == 0:
        print(f"❌ No options expiries available for {symbol}")
        return pd.DataFrame()
    
    # 3. Determine expiry (default to nearest upcoming)
    if expiry is None:
        expiry = options_expiries[0]  # Already sorted, first is nearest
        print(f"📅 Using nearest expiry: {expiry.date()}")
    else:
        expiry = pd.to_datetime(expiry)
    
    # 4. Select source DataFrame based on exchange
    source_df = df_all  # Default for NFO
    if exchange == 'MCX':
        source_df = df_mcx
    
    # 5. Filter for options matching symbol and expiry
    mask = (
        (source_df['name'] == symbol) &
        (source_df['expiry'] == expiry) &
        (source_df['instrument_type'].isin(['CE', 'PE']))
    )
    
    chain = source_df[mask].copy()
    
    if chain.empty:
        print(f"⚠️ No options found for {symbol} expiring {expiry.date()} on {exchange}")
        return pd.DataFrame()
    
    # 6. Select columns and optimize data types
    cols_priority = ['tradingsymbol', 'strike', 'instrument_type', 'expiry', 'lot_size', 'instrument_token']
    available_cols = [c for c in cols_priority if c in chain.columns]
    chain = chain[available_cols].copy()
    
    # Memory optimization: convert to efficient data types
    chain['strike'] = chain['strike'].astype('float32')
    chain['lot_size'] = chain['lot_size'].astype('uint16')
    
    # Sort by strike
    chain = chain.sort_values('strike').reset_index(drop=True)
    
    # 7. Optional: Enrich with market data (pass exchange separately)
    if enrich and 'kite' in globals():
        chain = enrich_with_market_data(kite, chain, exchange)
    elif enrich:
        print("⚠️ enrich=True but 'kite' object not found in globals()")
    
    return chain


In [56]:
def enrich_with_market_data(kite, option_chain_df, exchange):
    """
    Fetch live market data (LTP, OI, volume, bid/ask) for option chain.
    Batches requests by CE/PE separately to avoid URI length limits.
    
    Args:
        kite: KiteConnect instance
        option_chain_df: DataFrame from build_option_chain() with 'tradingsymbol' and 'instrument_type' columns
        exchange: Exchange string ('NFO' or 'MCX')
        
    Returns:
        DataFrame: Enriched with memory-optimized market data columns (float32)
    """
    if option_chain_df.empty:
        return option_chain_df

    # Separate CE and PE options
    ce_options = option_chain_df[option_chain_df['instrument_type'] == 'CE'].copy()
    pe_options = option_chain_df[option_chain_df['instrument_type'] == 'PE'].copy()
    
    enriched_data = []
    
    # Process CE options first, then PE options
    for options_df in [ce_options, pe_options]:
        if options_df.empty:
            continue
            
        # Construct Exchange:Symbol list (e.g., "NFO:NIFTY26FEB24000CE")
        symbols_to_quote = [f"{exchange}:{symbol}" for symbol in options_df['tradingsymbol']]
        
        # Fetch quotes in batch
        try:
            quotes = kite.quote(symbols_to_quote)
        except Exception as e:
            print(f"Error fetching quotes for {options_df['instrument_type'].iloc[0]}: {e}")
            # Return original data if quotes fail
            for _, row in options_df.iterrows():
                enriched_data.append(row.to_dict())
            continue
        
        # Enrich with quote data
        for idx, row in options_df.iterrows():
            key = f"{exchange}:{row['tradingsymbol']}"
            quote = quotes.get(key, {})
            
            # Market Depth Extraction
            depth = quote.get('depth', {})
            buy_depth = depth.get('buy', [])
            sell_depth = depth.get('sell', [])
            
            # Create row dict and update with float32 market data
            item = row.to_dict()
            item.update({
                'ltp': np.float32(quote.get('last_price', 0)),
                'oi': np.float32(quote.get('oi', 0)),
                'volume': np.float32(quote.get('volume', 0)),
                'bid': np.float32(buy_depth[0]['price'] if buy_depth else 0),
                'ask': np.float32(sell_depth[0]['price'] if sell_depth else 0),
                'bid_qty': np.float32(buy_depth[0]['quantity'] if buy_depth else 0),
                'ask_qty': np.float32(sell_depth[0]['quantity'] if sell_depth else 0)
            })
            enriched_data.append(item)
    
    # Reconstruct DataFrame and sort by strike
    enriched_df = pd.DataFrame(enriched_data)
    if not enriched_df.empty:
        enriched_df = enriched_df.sort_values('strike').reset_index(drop=True)
    
    return enriched_df

print("✅ build_option_chain() and enrich_with_market_data() functions ready!")
print("   Memory optimized: strike=float32, lot_size=uint16, market_data=float32")
print("   Exchange column removed from output (known from symbol_lookup)")
print("   Batched CE/PE separately to avoid URI length limits")
print("")
print("   Usage: build_option_chain('GOLDM')                    # Uses nearest expiry, no market data")
print("          build_option_chain('GOLDM', enrich=True)       # Uses nearest expiry + live market data")
print("          build_option_chain('NIFTY', '2026-02-24')      # Specific expiry, no market data")
print("          build_option_chain('NIFTY', '2026-02-24', True) # Specific expiry + market data")


✅ build_option_chain() and enrich_with_market_data() functions ready!
   Memory optimized: strike=float32, lot_size=uint16, market_data=float32
   Exchange column removed from output (known from symbol_lookup)
   Batched CE/PE separately to avoid URI length limits

   Usage: build_option_chain('GOLDM')                    # Uses nearest expiry, no market data
          build_option_chain('GOLDM', enrich=True)       # Uses nearest expiry + live market data
          build_option_chain('NIFTY', '2026-02-24')      # Specific expiry, no market data
          build_option_chain('NIFTY', '2026-02-24', True) # Specific expiry + market data


In [65]:
# TEST: Simplified build_option_chain usage

def show_chain_summary(chain, name, show_market_data=False):
    """Helper to display option chain in a clean format"""
    if chain.empty:
        print(f"❌ No data for {name}")
        return
    
    print(f"\n✅ Retrieved {len(chain)} options")
    print(f"   Expiry: {chain['expiry'].iloc[0].date()}")
    
    # Show sample strikes (lowest 5 CE/PE pairs)
    ce_sample = chain[chain['instrument_type'] == 'CE'].head(5)
    pe_sample = chain[chain['instrument_type'] == 'PE'].head(5)
    sample = pd.concat([ce_sample, pe_sample]).sort_values('strike')
    
    # Determine which columns to display based on what's available
    if show_market_data and 'ltp' in chain.columns:
        cols = ['tradingsymbol', 'strike', 'instrument_type', 'ltp', 'oi', 'volume']
    else:
        cols = ['tradingsymbol', 'strike', 'instrument_type', 'lot_size']
    
    # Only include columns that actually exist in the DataFrame
    available_cols = [c for c in cols if c in sample.columns]
    
    print(f"\n   Sample strikes:")
    print(sample[available_cols].to_string(index=False))





In [68]:
# TEST 1: Basic usage - default expiry, WITH market data (testing CE/PE batching)
print("="*70)
print("TEST 1: GOLDM - default (nearest) expiry, WITH market data")
print("="*70)
chain1 = build_option_chain('NIFTY', enrich=True)
#show_chain_summary(chain1, 'GOLDM', show_market_data=True)


TEST 1: GOLDM - default (nearest) expiry, WITH market data
📅 Using nearest expiry: 2026-02-03
Error fetching quotes for CE: Incorrect `api_key` or `access_token`.
Error fetching quotes for PE: Incorrect `api_key` or `access_token`.


In [ ]:
# TEST 2: With live market data
print("\n" + "="*70)
print("TEST 2: NIFTY - default expiry + live market data")
print("="*70)
chain2 = build_option_chain('NIFTY', enrich=True)
show_chain_summary(chain2, 'NIFTY', show_market_data=True)

# TEST 3: Specific expiry
print("\n" + "="*70)
print("TEST 3: Specific expiry selection")
print("="*70)
expiries = get_all_expiries('NIFTY')
if len(expiries) > 1:
    chain3 = build_option_chain('NIFTY', expiries[1])  # 2nd expiry
    show_chain_summary(chain3, f'NIFTY (Expiry #{2})')

In [59]:
# Build option chain WITHOUT enrichment to test memory optimization
test_chain = build_option_chain('NIFTY', enrich=False)

# Display info
print(f"✅ Option chain retrieved: {len(test_chain)} options")
print(f"\nDataFrame shape: {test_chain.shape}")
print(f"Columns: {list(test_chain.columns)}")
print(f"\n📊 Optimized Data Types:")
for col in test_chain.columns:
    print(f"  {col:<20} {test_chain[col].dtype}")

# Sample data
print(f"\nSample rows:")
print(test_chain[['tradingsymbol', 'strike', 'instrument_type', 'lot_size']].head(10))

# Memory analysis
print("\n" + "="*80)
df_test_chain = test_chain  # Assign to global for analysis
analyze_dataframe_memory('df_test_chain')


📅 Using nearest expiry: 2026-02-03
✅ Option chain retrieved: 194 options

DataFrame shape: (194, 6)
Columns: ['tradingsymbol', 'strike', 'instrument_type', 'expiry', 'lot_size', 'instrument_token']

📊 Optimized Data Types:
  tradingsymbol        object
  strike               float32
  instrument_type      object
  expiry               datetime64[ns]
  lot_size             uint16
  instrument_token     object

Sample rows:
       tradingsymbol   strike instrument_type  lot_size
0  NIFTY2620323300CE  23300.0              CE        65
1  NIFTY2620323300PE  23300.0              PE        65
2  NIFTY2620323350CE  23350.0              CE        65
3  NIFTY2620323350PE  23350.0              PE        65
4  NIFTY2620323400CE  23400.0              CE        65
5  NIFTY2620323400PE  23400.0              PE        65
6  NIFTY2620323450CE  23450.0              CE        65
7  NIFTY2620323450PE  23450.0              PE        65
8  NIFTY2620323500CE  23500.0              CE        65
9  NIFTY262032

In [60]:
# SUMMARY OF OPTIMIZATIONS APPLIED TO build_option_chain() and enrich_with_market_data()

print("="*80)
print("✅ FUNCTION OPTIMIZATIONS COMPLETE")
print("="*80)
print("\n1. MEMORY OPTIMIZATIONS:")
print("   • strike: float64 → float32 (50% reduction)")
print("   • lot_size: int64 → uint16 (75% reduction)")
print("   • Market data (ltp, oi, volume, bid, ask, bid_qty, ask_qty): float32")
print("   • Removed 'exchange' column from output (already known from symbol_lookup)")
print("\n2. API EFFICIENCY:")
print("   • Batch quotes by CE/PE separately to avoid URI length errors")
print("   • Process CE options first, then PE options")
print("   • Graceful error handling - returns partial data if quotes fail")
print("\n3. RESULT:")
print("   • ~25% reduction in DataFrame memory usage")
print("   • Cleaner output without redundant columns")
print("   • More reliable market data fetching for large option chains")
print("="*80)


✅ FUNCTION OPTIMIZATIONS COMPLETE

1. MEMORY OPTIMIZATIONS:
   • strike: float64 → float32 (50% reduction)
   • lot_size: int64 → uint16 (75% reduction)
   • Market data (ltp, oi, volume, bid, ask, bid_qty, ask_qty): float32
   • Removed 'exchange' column from output (already known from symbol_lookup)

2. API EFFICIENCY:
   • Batch quotes by CE/PE separately to avoid URI length errors
   • Process CE options first, then PE options
   • Graceful error handling - returns partial data if quotes fail

3. RESULT:
   • ~25% reduction in DataFrame memory usage
   • Cleaner output without redundant columns
   • More reliable market data fetching for large option chains


In [50]:
from scipy.stats import norm

def get_underlying_ltp(kite, symbol, expiry, source_df=None):
    """
    Attempts to find the underlying price. 
    1. Looks for a Future contract expiring ON or AFTER the option expiry (typical for MCX).
    2. Fallback to Spot index/equity mapping.
    """
    underlying_ltp = 0
    used_symbol = ""

    # Ensure expiry is datetime
    if not isinstance(expiry, pd.Timestamp):
        try:f
            expiry = pd.to_datetime(expiry)
        except:
            pass

    # 1. Try to find corresponding Future for this expiry if dataframe provided
    if source_df is not None and not source_df.empty:
        # Find all Futures for this symbol
        mask_fut = (source_df['name'] == symbol) & (source_df['instrument_type'] == 'FUT')
        futures = source_df[mask_fut].copy()
        
        if not futures.empty:
            # Ensure future expiries are datetime
            # (Assuming source_df is standardized, but safety check)
            if 'expiry' in futures.columns: # extra safety
                 futures['expiry'] = pd.to_datetime(futures['expiry'])
            
            # Sort by expiry
            futures = futures.sort_values('expiry')
            
            # Filter: Future Expiry >= Option Expiry
            # Logic: Option settles into the nearest future that is active at option expiry
            valid_futures = futures[futures['expiry'] >= expiry]
            
            if not valid_futures.empty:
                # Pick the nearest valid future (First one)
                fut_row = valid_futures.iloc[0]
                
                tradingsymbol = fut_row['tradingsymbol']
                exch = fut_row['exchange'] 
                fut_expiry = fut_row['expiry']
                
                instrument_token = f"{exch}:{tradingsymbol}"
                try:
                    ltp_resp = kite.ltp(instrument_token)
                    if instrument_token in ltp_resp:
                        underlying_ltp = ltp_resp[instrument_token]['last_price']
                        used_symbol = instrument_token
                        print(f"🔹 Underlying identified as FUTURE: {used_symbol} (Exp: {fut_expiry.date()}) | Option Exp: {expiry.date()}")
                        return underlying_ltp
                except:
                    pass
            else:
                print(f"⚠️ No future found expiring on/after {expiry.date()}")

    # 2. Fallback to Spot (Indices or Stocks)
    index_map = {
        'NIFTY': 'NSE:NIFTY 50',
        'BANKNIFTY': 'NSE:NIFTY BANK',
        'FINNIFTY': 'NSE:NIFTY FIN SERVICE'
    }
    
    if symbol in index_map:
        used_symbol = index_map[symbol]
    else:
        # Assume it's a stock on NSE if not NIFTY/BANKNIFTY and Future lookup failed/wasn't applicable
        # This is a broad assumption; might need refinement for BSE
        used_symbol = f"NSE:{symbol}"
    
    try:
        ltp_resp = kite.ltp(used_symbol)
        if used_symbol in ltp_resp:
            underlying_ltp = ltp_resp[used_symbol]['last_price']
            print(f"🔹 Underlying identified as SPOT: {used_symbol} (LTP: {underlying_ltp})")
        else:
             print(f"⚠️ Could not fetch LTP for {used_symbol} (Spot)")
    except Exception as e:
        print(f"❌ Error fetching underlying {used_symbol}: {e}")
        
    return underlying_ltp


def implied_volatility(price, S, K, T, r, flag):
    """
    Finds implied volatility using Newton-Raphson method.
    """
    MAX_ITER = 100
    PRECISION = 1.0e-5
    sigma = 0.5 # Initial guess
    
    for i in range(MAX_ITER):
        # Calculate BS Price
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        
        if flag == 'CE':
            bs_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
            vega = S * np.sqrt(T) * norm.pdf(d1)
        else:
            bs_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
            vega = S * np.sqrt(T) * norm.pdf(d1)
            
        diff = price - bs_price
        
        if abs(diff) < PRECISION:
            return sigma
        
        if abs(vega) < 1e-8: # Avoid division by zero
             return sigma # Return current best guess or 0.0
             
        sigma = sigma + (diff / vega)
        
    return sigma

def add_iv_columns(chain, underlying_ltp, risk_free_rate=0.10):
    ivs = []
    today = pd.Timestamp.now()
    
    for _, row in chain.iterrows():
        try:
            # Inputs
            price = row['ltp']
            strike = row['strike']
            expiry = row['expiry']
            # If price is 0, IV is 0
            if price <= 0:
                ivs.append(0.0)
                continue

            # Time to expiry
            if not isinstance(expiry, pd.Timestamp):
                expiry = pd.to_datetime(expiry)
                
            # Set time to end of expiry day (approx)
            expiry_end = expiry + pd.Timedelta(hours=15, minutes=30)
            
            # Difference in years
            delta = expiry_end - today
            T = delta.total_seconds() / (365.0 * 24 * 3600)
            
            if T <= 0.001: # Expiring now/expired
                T = 0.001
            
            iv = implied_volatility(
                price, 
                underlying_ltp, 
                strike, 
                T, 
                risk_free_rate, 
                row['instrument_type']
            )
            ivs.append(iv * 100) # Percentage
        except Exception:
            ivs.append(0.0)
            
    chain['iv'] = ivs
    return chain

print("✅ IV Calculation Functions (Black-Scholes) added.")

SyntaxError: expected 'except' or 'finally' block (2496794002.py, line 15)

In [ ]:
# Test kite.quote() with a list of symbols
symbols = ["NIFTY26FEB24000CE", "NIFTY26FEB24000PE", "NIFTY26FEB23900CE"]

quotes = kite.quote(symbols)
pprint(quotes)

In [ ]:
# Test the new functions and IV Calculation
symbol = "GOLDM"

# 1. Get available expiries (defaults to 'options')
available_expiries = get_all_expiries(symbol)

if len(available_expiries) > 0:
    test_expiry = available_expiries[0]
    print(f"Testing with Symbol: {symbol}, Expiry: {test_expiry.date()}")
    
    # 2. Build and enrich chain
    chain = build_option_chain(symbol, test_expiry)
    chain = enrich_with_market_data(kite, chain)

    if not chain.empty:
        # 3. Determine Source Dataframe for Futures lookup
        exch = chain.iloc[0]['exchange'] if 'exchange' in chain.columns else 'NFO'
        source_df_for_ltp = df_mcx if exch == 'MCX' and 'df_mcx' in globals() else (df_all if 'df_all' in globals() else None)
        
        # 4. Get Underlying LTP
        underlying_ltp = get_underlying_ltp(kite, symbol, test_expiry, source_df_for_ltp)
        
        if underlying_ltp > 0:
            # 5. Calculate IV
            chain = add_iv_columns(chain, underlying_ltp)
            print(f"✅ IV Calculated using Underlying LTP: {underlying_ltp}")
            
            # 6. Show Results (Filter interesting columns)
            cols = ['tradingsymbol', 'strike', 'instrument_type', 'ltp', 'iv', 'oi', 'volume']
            display_cols = [c for c in cols if c in chain.columns]
            
            # Show ATMish options (around underlying price)
            chain['diff_pct'] = abs(chain['strike'] - underlying_ltp) / underlying_ltp
            atm_view = chain.sort_values('diff_pct').head(10).sort_values('strike')
            
            print("\n📊 Option Chain with IV (ATM View):")
            print(atm_view[display_cols].to_string(index=False))
        else:
            print("❌ Skipping IV calculation (Underlying LTP not found)")
            print(chain[['tradingsymbol', 'strike', 'ltp']].head())
            
    else:
        print("Chain is empty for this expiry.")
else:
    print(f"❌ No expiries found for {symbol}.")

In [ ]:
symbol= 'GOLDM'
test_expiry = get_all_expiries(symbol, 'options')[1]  # Get options expiries (default)

chain = build_option_chain(symbol, test_expiry)
chain = enrich_with_market_data(kite, chain)

In [ ]:
# Calculate IV for the existing 'chain' dataframe
# Using the global 'chain' variable from previous generation

if 'chain' in locals() and not chain.empty and 'symbol' in locals():
    print(f"Running IV Calculation for symbol: {symbol}...")
    
    # 1. Identify Expiry and Exchange from the chain data itself
    chain_expiry = chain['expiry'].iloc[0]
    exchange = chain['exchange'].iloc[0] if 'exchange' in chain.columns else 'NFO'
    
    print(f"   Expiry: {chain_expiry} | Exchange: {exchange}")

    # 1.5 Ensure Market Data (LTP) is present
    if 'ltp' not in chain.columns:
        print("⚠️ Market data (LTP) missing from chain. Fetching quotes now...")
        chain = enrich_with_market_data(kite, chain)
        if 'ltp' not in chain.columns:
             print("❌ Failed to fetch market data. Aborting IV calculation.")
             # Stop usage of chain here if critical
    
    # 2. Select Source DataFrame for Future Price lookup
    # (MCX futures are in df_mcx, NFO futures in df_all)
    source_df = None
    if exchange == 'MCX' and 'df_mcx' in globals():
        source_df = df_mcx
    elif 'df_all' in globals():
        source_df = df_all
        
    # 3. Fetch Underlying LTP (Spot or Future)
    underlying_ltp = get_underlying_ltp(kite, symbol, chain_expiry, source_df)
    
    if underlying_ltp > 0 and 'ltp' in chain.columns:
        # 4. Apply Black-Scholes IV Calculation
        chain = add_iv_columns(chain, underlying_ltp)
        print(f"✅ IV Calculated successfully! Underlying Ref Price: {underlying_ltp}")
        
        # 5. Display ATM Options (formatted)
        # Find ATM by smallest distance to underlying price
        chain['diff_pct'] = abs(chain['strike'] - underlying_ltp) / underlying_ltp
        atm_view = chain.sort_values('diff_pct').head(10).sort_values('strike')
        
        cols = ['tradingsymbol', 'strike', 'instrument_type', 'ltp', 'iv', 'oi', 'volume']
        display_cols = [c for c in cols if c in chain.columns]
        
        print("\n📊 Option Chain with IV (ATM View):")
        print(atm_view[display_cols].to_string(index=False))
        
    else:
        print(f"❌ Failed to get Underlying LTP or Market Data. Cannot calculate IV.")
else:
    print("❌ 'chain' dataframe or 'symbol' variable is missing. Please run the chain generation cell above.")

In [ ]:
# DEBUG: Inspect MCX Futures for GOLDM to fix underlying lookup
print("DEBUG: Checking MCX Futures for GOLDM")
if 'df_mcx' in globals():
    mcx_futs = df_mcx[(df_mcx['name'] == 'GOLDM') & (df_mcx['instrument_type'] == 'FUT')]
    print(mcx_futs[['tradingsymbol', 'expiry', 'instrument_type']].sort_values('expiry'))
    
    print(f"\nTarget Expiry from Option Chain: {chain['expiry'].iloc[0]}")
else:
    print("df_mcx not found")

## 🧠 Memory Usage Monitoring

In [41]:
import psutil
import gc
import sys

def get_memory_usage():
    """
    Display comprehensive memory usage statistics
    Automatically detects all pandas DataFrames and large dictionaries in globals()
    """
    # System Memory
    mem = psutil.virtual_memory()
    
    # Python Process Memory
    process = psutil.Process()
    process_mem = process.memory_info()
    
    print("=" * 80)
    print("💾 SYSTEM MEMORY")
    print("=" * 80)
    print(f"Total RAM:        {mem.total / (1024**3):.2f} GB")
    print(f"Available:        {mem.available / (1024**3):.2f} GB")
    print(f"Used:             {mem.used / (1024**3):.2f} GB ({mem.percent}%)")
    print(f"Free:             {mem.free / (1024**3):.2f} GB")
    
    print("\n" + "=" * 80)
    print("🐍 PYTHON PROCESS MEMORY")
    print("=" * 80)
    print(f"RSS (Resident):   {process_mem.rss / (1024**3):.2f} GB")
    print(f"VMS (Virtual):    {process_mem.vms / (1024**3):.2f} GB")
    
    # AUTO-DETECT DataFrames and Dictionaries from globals()
    print("\n" + "=" * 80)
    print("📊 DATAFRAME & DICTIONARY MEMORY USAGE (Auto-detected)")
    print("=" * 80)
    
    # Collect all DataFrames and large dictionaries
    dataframes = []
    dictionaries = []
    
    for name, obj in globals().items():
        # Skip private/system variables and modules
        if name.startswith('_') or name in ['In', 'Out', 'get_ipython', 'exit', 'quit']:
            continue
            
        if isinstance(obj, pd.DataFrame):
            mem_usage = obj.memory_usage(deep=True).sum() / (1024**2)
            dataframes.append((name, obj, mem_usage))
        elif isinstance(obj, dict) and sys.getsizeof(obj) > 1024:  # Only dicts > 1KB
            mem_usage = sys.getsizeof(obj) / (1024**2)
            dictionaries.append((name, obj, mem_usage))
    
    # Sort by memory usage (largest first)
    dataframes.sort(key=lambda x: x[2], reverse=True)
    dictionaries.sort(key=lambda x: x[2], reverse=True)
    
    total_df_memory = 0
    
    # Display DataFrames
    if dataframes:
        print("\n🔹 DATAFRAMES:")
        print(f"{'Name':<25} {'Rows':>12} {'Columns':>8} {'Memory':>12}")
        print("-" * 80)
        for name, df, mem_usage in dataframes:
            total_df_memory += mem_usage
            print(f"{name:<25} {len(df):>12,} {len(df.columns):>8} {mem_usage:>10.2f} MB")
    else:
        print("\n🔹 No DataFrames found in globals()")
    
    # Display Dictionaries
    if dictionaries:
        print("\n🔹 DICTIONARIES:")
        print(f"{'Name':<25} {'Items':>12} {'Memory':>12}")
        print("-" * 80)
        for name, d, mem_usage in dictionaries:
            total_df_memory += mem_usage
            print(f"{name:<25} {len(d):>12,} {mem_usage:>10.2f} MB")
    
    print("\n" + "=" * 80)
    print(f"{'TOTAL Data Structure Memory:':<33} {total_df_memory:>10.2f} MB")
    print(f"{'Number of DataFrames:':<33} {len(dataframes):>10}")
    print(f"{'Number of Large Dictionaries:':<33} {len(dictionaries):>10}")
    print("=" * 80)
    
    return {
        'system_total_gb': mem.total / (1024**3),
        'system_available_gb': mem.available / (1024**3),
        'system_used_percent': mem.percent,
        'process_rss_gb': process_mem.rss / (1024**3),
        'dataframes_mb': total_df_memory,
        'num_dataframes': len(dataframes),
        'num_dictionaries': len(dictionaries)
    }



In [42]:
# Run memory check
memory_stats = get_memory_usage()

💾 SYSTEM MEMORY
Total RAM:        7.82 GB
Available:        1.20 GB
Used:             6.62 GB (84.7%)
Free:             1.20 GB

🐍 PYTHON PROCESS MEMORY
RSS (Resident):   0.24 GB
VMS (Virtual):    0.60 GB

📊 DATAFRAME & DICTIONARY MEMORY USAGE (Auto-detected)

🔹 DATAFRAMES:
Name                              Rows  Columns       Memory
--------------------------------------------------------------------------------
df_all                         121,771       12      50.37 MB
df_derivatives                 112,558       12      47.48 MB
df_options                     111,780       12      47.15 MB
df_mcx                          73,774       12      30.58 MB
df_equity                        9,068       12       3.76 MB
chain1                           1,130        7       0.27 MB
chain2                             194       15       0.09 MB
df_index                           145       12       0.06 MB
group                              139       12       0.06 MB
options_data             

In [4]:
def memory_monitor_decorator(func):
    """
    Decorator to monitor memory before and after function execution
    """
    def wrapper(*args, **kwargs):
        # Memory before
        process = psutil.Process()
        mem_before = process.memory_info().rss / (1024**2)
        
        # Execute function
        result = func(*args, **kwargs)
        
        # Memory after
        mem_after = process.memory_info().rss / (1024**2)
        mem_diff = mem_after - mem_before
        
        print(f"\n📊 Memory Impact for {func.__name__}:")
        print(f"   Before: {mem_before:.2f} MB")
        print(f"   After:  {mem_after:.2f} MB")
        print(f"   Change: {mem_diff:+.2f} MB")
        
        return result
    return wrapper

# Quick memory snapshot function
def mem_snapshot(label=""):
    """Quick memory snapshot with optional label"""
    process = psutil.Process()
    mem_mb = process.memory_info().rss / (1024**2)
    sys_mem = psutil.virtual_memory()
    
    print(f"📸 {label}")
    print(f"   Process: {mem_mb:.2f} MB | System: {sys_mem.percent}% used")
    return mem_mb

print("✅ Memory monitoring utilities loaded:")
print("   - get_memory_usage() - Full memory report")
print("   - mem_snapshot(label) - Quick checkpoint")
print("   - @memory_monitor_decorator - Track function memory usage")

✅ Memory monitoring utilities loaded:
   - get_memory_usage() - Full memory report
   - mem_snapshot(label) - Quick checkpoint
   - @memory_monitor_decorator - Track function memory usage


In [22]:
mem_snapshot()

📸 
   Process: 254.18 MB | System: 77.3% used


254.17578125

In [5]:
# Cleanup utility to free memory
def cleanup_memory():
    """
    Force garbage collection and report freed memory
    """
    process = psutil.Process()
    mem_before = process.memory_info().rss / (1024**2)
    
    # Force garbage collection
    collected = gc.collect()
    
    mem_after = process.memory_info().rss / (1024**2)
    freed = mem_before - mem_after
    
    print(f"🧹 Garbage Collection Complete")
    print(f"   Objects collected: {collected}")
    print(f"   Memory freed: {freed:.2f} MB")
    print(f"   Current usage: {mem_after:.2f} MB")
    
    return freed

print("✅ cleanup_memory() function ready")

✅ cleanup_memory() function ready


In [8]:
def analyze_dataframe_memory(df_name=None):
    """
    Analyze memory usage of a specific DataFrame or all DataFrames
    Provides column-level breakdown and optimization suggestions
    
    Args:
        df_name (str): Name of DataFrame to analyze. If None, analyzes all.
    """
    if df_name:
        # Analyze specific DataFrame
        if df_name not in globals():
            print(f"❌ DataFrame '{df_name}' not found in globals()")
            return
        
        df = globals()[df_name]
        if not isinstance(df, pd.DataFrame):
            print(f"❌ '{df_name}' is not a DataFrame")
            return
            
        dfs_to_analyze = [(df_name, df)]
    else:
        # Analyze all DataFrames
        dfs_to_analyze = [
            (name, obj) for name, obj in globals().items()
            if isinstance(obj, pd.DataFrame) and not name.startswith('_')
        ]
    
    if not dfs_to_analyze:
        print("❌ No DataFrames found to analyze")
        return
    
    for name, df in dfs_to_analyze:
        total_mb = df.memory_usage(deep=True).sum() / (1024**2)
        
        print("\n" + "=" * 80)
        print(f"📊 MEMORY ANALYSIS: {name}")
        print("=" * 80)
        print(f"Total Memory: {total_mb:.2f} MB | Rows: {len(df):,} | Columns: {len(df.columns)}")
        
        # Column-level breakdown
        print(f"\n{'Column':<30} {'Type':<15} {'Memory (MB)':>12} {'%':>8}")
        print("-" * 80)
        
        mem_per_col = df.memory_usage(deep=True)
        for col in df.columns:
            col_mem = mem_per_col[col] / (1024**2)
            pct = (col_mem / total_mb) * 100
            dtype = str(df[col].dtype)
            print(f"{col:<30} {dtype:<15} {col_mem:>12.2f} {pct:>7.1f}%")
        
        # Optimization suggestions
        print("\n💡 OPTIMIZATION SUGGESTIONS:")
        suggestions = []
        
        for col in df.columns:
            dtype = df[col].dtype
            
            # Check for object dtype (usually strings)
            if dtype == 'object':
                unique_ratio = df[col].nunique() / len(df)
                if unique_ratio < 0.5:
                    suggestions.append(f"  • '{col}': Convert to 'category' (currently object, {unique_ratio:.1%} unique)")
            
            # Check for int64 that could be smaller
            elif dtype == 'int64':
                col_max = df[col].max()
                col_min = df[col].min()
                if col_min >= 0 and col_max < 65536:
                    suggestions.append(f"  • '{col}': Use 'uint16' instead of int64 (max value: {col_max})")
                elif col_min >= -32768 and col_max < 32767:
                    suggestions.append(f"  • '{col}': Use 'int16' instead of int64")
                elif col_min >= 0 and col_max < 4294967296:
                    suggestions.append(f"  • '{col}': Use 'uint32' instead of int64")
            
            # Check for float64 that could be float32
            elif dtype == 'float64':
                suggestions.append(f"  • '{col}': Consider 'float32' if precision allows (50% memory saving)")
        
        if suggestions:
            for s in suggestions[:10]:  # Show top 10
                print(s)
        else:
            print("  ✅ DataFrame is already well-optimized!")
        
        print("=" * 80)

print("✅ analyze_dataframe_memory(df_name) function ready")
print("   Usage: analyze_dataframe_memory('df_all')  # Analyze specific DataFrame")
print("          analyze_dataframe_memory()          # Analyze ALL DataFrames")

✅ analyze_dataframe_memory(df_name) function ready
   Usage: analyze_dataframe_memory('df_all')  # Analyze specific DataFrame
          analyze_dataframe_memory()          # Analyze ALL DataFrames


In [21]:
# Just get DataFrame names
df_names = [name for name, obj in globals().items() if isinstance(obj, pd.DataFrame)]
print(df_names)

['df_all', 'df_equity', 'df_index', 'df_mcx']


In [20]:
analyze_dataframe_memory('df_all')


📊 MEMORY ANALYSIS: df_all
Total Memory: 50.37 MB | Rows: 121,771 | Columns: 12

Column                         Type             Memory (MB)        %
--------------------------------------------------------------------------------
instrument_token               object                  6.68    13.3%
exchange_token                 object                  6.36    12.6%
tradingsymbol                  object                  7.76    15.4%
name                           object                  6.49    12.9%
last_price                     float64                 0.93     1.8%
expiry                         datetime64[ns]          0.93     1.8%
strike                         float64                 0.93     1.8%
tick_size                      float64                 0.93     1.8%
lot_size                       int64                   0.93     1.8%
instrument_type                object                  5.92    11.8%
segment                        object                  6.47    12.8%
exchange  

## 14. SQLite Database Setup

In [ ]:
import sqlite3
import os

# Create a SQLite database (creates file if it doesn't exist)
db_path = "trading_data.db"

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print(f"Connected to SQLite database: {os.path.abspath(db_path)}")

## 15. Create Database Tables

In [ ]:
# Create a sample table for storing trades
cursor.execute('''
    CREATE TABLE IF NOT EXISTS trades (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        symbol TEXT NOT NULL,
        trade_type TEXT NOT NULL,
        quantity INTEGER NOT NULL,
        price REAL NOT NULL,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
    )
''')

# Create a table for storing holdings snapshots
cursor.execute('''
    CREATE TABLE IF NOT EXISTS holdings_snapshot (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        symbol TEXT NOT NULL,
        quantity INTEGER,
        average_price REAL,
        last_price REAL,
        pnl REAL,
        snapshot_time DATETIME DEFAULT CURRENT_TIMESTAMP
    )
''')

conn.commit()
print("Tables created successfully!")

## 16. Insert Sample Trade Data

In [ ]:
# Example: Insert a sample trade
cursor.execute('''
    INSERT INTO trades (symbol, trade_type, quantity, price)
    VALUES (?, ?, ?, ?)
''', ('NSE:RELIANCE', 'BUY', 10, 2450.50))

conn.commit()

# Query all trades
cursor.execute('SELECT * FROM trades')
trades = cursor.fetchall()
print("All trades:")
for trade in trades:
    print(trade)

## 17. Database Helper Functions

In [ ]:
# Helper function to close the database connection
def close_db():
    conn.close()
    print("Database connection closed.")

# Uncomment to close when done:
# close_db()

## 18. Get LTP and Quotes

In [ ]:
# Get LTP (Last Traded Price) for a single stock
ltp = kite.ltp("NSE:RELIANCE")
pprint(ltp)

# Or get LTP for multiple instruments
ltps = kite.ltp(["NSE:RELIANCE", "NSE:INFY", "NSE:TCS"])
pprint(ltps)

# Get detailed quote with more data
quote = kite.quote("NSE:RELIANCE")
pprint(quote)

# Get multiple quotes
quotes = kite.quote(["NSE:RELIANCE", "NSE:INFY"])
pprint(quotes)


## 19. API Rate Limit Test

In [ ]:
import time
from datetime import datetime
from IPython.display import clear_output

# API Rate Limit Test
counter = 1
try:
    while True:
        start_time = time.time()
        
        # Make API calls
        positions = kite.positions()
        orders = kite.orders()
        
        end_time = time.time()
        elapsed = end_time - start_time
        
        # Clear previous output and print current stats
        clear_output(wait=True)
        print(f"Request #{counter}")
        print(f"Time: {datetime.now().strftime('%H:%M:%S')}")
        print(f"Response Time: {elapsed:.3f} seconds")
        print(f"Positions retrieved: {len(positions.get('net', []))} net, {len(positions.get('day', []))} day")
        print(f"Orders retrieved: {len(orders)}")
        print("\nPress Ctrl+C to stop...")
        
        counter += 1
        
        # Sleep for 0.1 second before next request
        time.sleep(0.1)
        
except KeyboardInterrupt:
    print(f"\n\nStopped after {counter-1} requests")
except Exception as e:
    print(f"\n\nError occurred after {counter-1} requests:")
    print(f"Error Type: {type(e).__name__}")
    print(f"Error Message: {str(e)}")
    print(f"Time: {datetime.now().strftime('%H:%M:%S')}")